# Web Architecture

_This notebook demonstrates how a server can now serve the client with predictions, and also with real-time data coming in through the app that the user is tracking metrics for._

_The user receives the data from the server and then can track it on their own device._

## Imports

In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import multiprocessing
import json
import pickle
import random
import time
import datetime
from IPython.display import IFrame

## Server

In [76]:
# Variables
DOMAIN = 'localhost'
PORT = 9090
URL = 'localhost:{port}'.format(port=PORT)

class S(BaseHTTPRequestHandler):
    def _set_response_html(self):
        self.send_response(200)
        self.send_header("Access-Control-Allow-Origin", "*")
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
    def _set_response_json(self):
        self.send_response(200)
        self.send_header("Access-Control-Allow-Origin", "*")
        self.send_header('Content-type', 'application/json')
        self.end_headers()

    def do_GET(self):
        logging.info("GET request,\nPath: %s\nHeaders:\n%s\n", 
                     str(self.path), str(self.headers))
        self._set_response_html()
        self.wfile.write("GET request for {}\n".\
                         format(self.path).encode('utf-8'))
        
    def do_POST(self):
        # Get the size of data
        content_length = int(self.headers['Content-Length'])
        # Get the data itself
        post_data = self.rfile.read(content_length)
        logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     str(self.path), str(self.headers), 
                     json.loads(post_data.decode('utf-8')))
        
        msg = self.gen_reply(json.loads(post_data.decode('utf-8')))
        
        self._set_response_json()
        self.wfile.write(json.dumps(msg).encode('utf-8'))
        
    def gen_reply(self, obj):         
        msg = {'entity': 'server'}
        if obj['subj'] == 'handshake':
            msg['subj'] = 'handshake'
            with open('observations_'+obj['event_name']+'.pkl', 'rb') as f:    
                data = pickle.load(f)
            with open('predictions_'+obj['event_name']+'_2000.pkl', 'rb') as f:    
                fcst = pickle.load(f)
            msg['ds'] = [datetime.datetime.timestamp(x) for x in fcst.ds] # datestamps
            msg['y'] = [y for y in data.y] # observations
            msg['yhat'] = [yhat for yhat in fcst.yhat] # predictions
            
        elif obj['subj'] == 'polling':
            with open('predictions_'+obj['event_name']+'_2000.pkl', 'rb') as f:    
                fcst = pickle.load(f)
            msg['subj'] = 'polling'
            msg['ds'] = datetime.datetime.timestamp(datetime.datetime.now())
            msg['y'] = random.choice([yhat for yhat in fcst.yhat])
            
        return msg
        
def run(server_class=HTTPServer, handler_class=S, port=8080):
    logging.basicConfig(level=logging.INFO)
    server_address = (DOMAIN, PORT)
    httpd = server_class(server_address, handler_class)
    logging.info('Starting httpd at <%s>...\n'%URL)
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()
    logging.info('Stopping httpd...\n')

# start the server as a separate process
server_process = multiprocessing.Process(target=run)
server_process.daemon = True
server_process.start()

INFO:root:Starting httpd at <localhost:9090>...



## Client

The client can now track the live event count for the event named **_fL_**, and compare it to the predicted numbers, as well as see historical figures for the metric;

In [77]:
IFrame(src='web/client.html', width=700, height=320)

## Server Shutdown

In [82]:
# stop the server
server_process.terminate()